In [1]:
import json
import numpy as np
import pandas as pd
from time import time

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import pickle 

/tmp/ipykernel_138095/923370504.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
name_model = "mlp_rnd_walk_dist-micro_publaynet_50"
with open(f"./dataset/json/{name_model}.json", "r") as f:
    dataset = json.load(f)

In [3]:
print("Первый x:\n", dataset["x"][0])
print("Последний, x:\n", dataset["x"][-1])
print("Размер x:\t", len(dataset["x"][0]))
print("Первый y:\t", dataset["y"][0])
print("Последний y:\t", dataset["y"][0])
print("Кол-во записей:\t", len(dataset["x"]))

Первый x:
 [0.010723860589812333, 0.010723860589812333, 0.0, 0.0, 0.010723860589812333, 0.005361930294906166, 0.0, 0.0, 0.0, 0.005361930294906166, 0.005361930294906166, 0.002680965147453083, 0.005361930294906166, 0.0, 0.0, 0.005361930294906166, 0.010723860589812333, 0.040214477211796246, 0.09115281501340483, 0.09115281501340483, 0.09115281501340483, 0.09115281501340483, 0.040214477211796246, 0.010723860589812333, 0.010723860589812333, 0.010723860589812333, 0.03485254691689008, 0.03485254691689008, 0.1126005361930295, 0.00804289544235925, 0.04825737265415549, 0.00804289544235925, 0.010723860589812333, 0.005361930294906166, 0.005361930294906166, 0.013404825737265416, 0.010723860589812333, 0.010723860589812333, 0.010723860589812333, 0.013404825737265416, 0.013404825737265416, 0.013404825737265416, 0.00804289544235925, 0.00804289544235925, 0.010723860589812333, 0.040214477211796246, 0.00804289544235925, 0.005361930294906166, 0.00804289544235925, 0.00804289544235925]
Последний, x:
 [0.0, 0.

In [4]:

X = np.array(dataset["x"])
y = np.array(dataset["y"])


In [5]:
# y = y[X.sum(1)!=0]
# X = X[X.sum(1)!=0, :]


In [6]:
# norm_vec = lambda n_vec: n_vec/n_vec.sum() if n_vec.sum() != 0 else n_vec 
# X = np.array([norm_vec(x) for x in X])
# X

In [7]:
len(y), len(X)

(500, 500)

In [8]:
keys = np.unique(y)
Y_keys = dict()
for i, key in enumerate(keys):
    zeros = np.zeros((len(keys)))
    zeros[i] = 1
    Y_keys[key] = zeros
Y_keys
Y = np.array([Y_keys[yi] for yi in y] )
Y_keys, Y

({1: array([1., 0., 0., 0., 0.]),
  2: array([0., 1., 0., 0., 0.]),
  3: array([0., 0., 1., 0., 0.]),
  4: array([0., 0., 0., 1., 0.]),
  5: array([0., 0., 0., 0., 1.])},
 array([[1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        ...,
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.]]))

In [9]:
X_clus_list = []
Y_clus_list = []
for i, item in enumerate(Y_keys.keys()):
    X_clus_list.append(X[Y[:,i]==1,:])
    Y_clus_list.append(Y[Y[:,i]==1,:])

In [10]:
balans_len = [len(yc) for yc in Y_clus_list]
min_len = min(balans_len)
min_len, balans_len

(100, [100, 100, 100, 100, 100])

In [11]:
X = np.concatenate([xc[:min_len] for xc in X_clus_list])
Y = np.concatenate([yc[:min_len] for yc in Y_clus_list])

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [58]:
i = 0
aas = [0.1, 0.01, 0.001, 0.0001, 0.00001]
miis = [200, 500, 1000, 2000, 5000, 10000]
modds = [(20,), (20, 20), (20, 20, 20, 20), (50, ), (50, 50), ( 50, 50, 50, 50), (100,), (100, 100), (100, 100, 100, 100), (20, 50, 100), (100, 50, 20)]
df = pd.DataFrame(columns = ['alpha','max_iter','model', 'train_acc','test_acc','train_time'])
for a in aas:
    for mi in miis:
        for mod in modds:
            print(i)
            st = time()
            mlp = MLPClassifier(hidden_layer_sizes=mod, alpha=a, max_iter=mi)
            mlp.fit(X_train, y_train)
            end = time() - st
            
            acc_tr = accuracy_score(y_train, mlp.predict(X_train)) # Train Accuracy
            acc = accuracy_score(y_test, mlp.predict(X_test)) # Test Accuracy
            df.loc[i] = [a, mi, mod, acc_tr,acc,end]
            i=i+1

0
1


/home/daniil/program/python/project/layout_highlighter/imgDoc/env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/daniil/program/python/project/layout_highlighter/imgDoc/env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


2


/home/daniil/program/python/project/layout_highlighter/imgDoc/env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


3


...

312
313
314


/home/daniil/program/python/project/layout_highlighter/imgDoc/env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5000) reached and the optimization hasn't converged yet.
  warnings.warn(


315
316
317
318
319


/home/daniil/program/python/project/layout_highlighter/imgDoc/env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10000) reached and the optimization hasn't converged yet.
  warnings.warn(


320
321
322
323
324
325
326
327
328
329


In [59]:
df # Results
# 0	0.00001	2000	(50, 50, 50, 50)	0.765271	0.187783	9.691846 (43)
# 	0.00001	2000	(50, 50, 50, 50)	0.75	0.192308	9.430116 (44)
# 0.00001	2000	(50, 50, 50, 50)	0.616516	0.199095	10.068232 (45)
# 0.00001	2000	(50, 50, 50, 50)	0.796946	0.20362	6.908683 (46)

,alpha,max_iter,model,train_acc,test_acc,train_time
0,0.10000,200,"(20,)",0.0000,0.00,0.159025
1,0.10000,200,"(20, 20)",0.0000,0.00,0.233113
2,0.10000,200,"(20, 20, 20, 20)",0.0000,0.00,0.292420
3,0.10000,200,"(50,)",0.0000,0.00,0.253217
4,0.10000,200,"(50, 50)",0.0075,0.00,0.445111
...,...,...,...,...,...,...
325,0.00001,10000,"(100,)",0.9125,0.11,8.191900
326,0.00001,10000,"(100, 100)",0.9125,0.13,3.618923
327,0.00001,10000,"(100, 100, 100, 100)",0.9025,0.22,2.354440
328,0.00001,10000,"(20, 50, 100)",0.9075,0.15,4.584880


In [60]:
df["train_acc"].max()
df[df["train_acc"] == df["train_acc"].max()]

,alpha,max_iter,model,train_acc,test_acc,train_time
162,0.001,1000,"(100, 100, 100, 100)",0.945,0.25,2.72224


In [61]:
df[df["test_acc"] == df["test_acc"].max()]

,alpha,max_iter,model,train_acc,test_acc,train_time
30,0.100,1000,"(100, 100, 100, 100)",0.900,0.25,1.909127
162,0.001,1000,"(100, 100, 100, 100)",0.945,0.25,2.722240


In [62]:
df.sort_values("test_acc",  ascending=False)[:15]

,alpha,max_iter,model,train_acc,test_acc,train_time
162,0.00100,1000,"(100, 100, 100, 100)",0.9450,0.25,2.722240
30,0.10000,1000,"(100, 100, 100, 100)",0.9000,0.25,1.909127
327,0.00001,10000,"(100, 100, 100, 100)",0.9025,0.22,2.354440
53,0.10000,5000,"(20, 50, 100)",0.8700,0.22,4.933351
151,0.00100,500,"(100, 100, 100, 100)",0.9125,0.21,2.198701
261,0.00010,10000,"(100, 100, 100, 100)",0.9125,0.21,2.359888
93,0.01000,1000,"(50, 50, 50, 50)",0.9100,0.21,2.153414
41,0.10000,2000,"(100, 100, 100, 100)",0.9400,0.21,2.026403
194,0.00100,10000,"(100, 100)",0.9125,0.21,4.046854
42,0.10000,2000,"(20, 50, 100)",0.7725,0.21,4.594693


In [101]:
df[df["test_acc"] == df["test_acc"].max()]

,alpha,max_iter,model,train_acc,test_acc,train_time
169,1.0,500,"(20, 20, 20, 20, 20)",0.816742,0.377828,1.473909


In [136]:
df[df["test_acc"] == df["test_acc"].max()]

,alpha,max_iter,model,train_acc,test_acc,train_time
34,0.001,20000,"(50, 5, 10, 15, 20)",0.44457,0.192308,11.896763


In [18]:
max_acc = 0.22
for i in range(100):
    mlp = MLPClassifier(hidden_layer_sizes=(50, 100, 50, 20), alpha=0.001, max_iter=500)
    mlp.fit(X_train, y_train)
    
    
    acc_tr = accuracy_score(y_train, mlp.predict(X_train)) # Train Accuracy
    acc = accuracy_score(y_test, mlp.predict(X_test)) # Test Accuracy
    print(f"{i}"+" "*10, end="\r")
    if max_acc < acc:
        max_acc = acc
        print(acc_tr, acc)
        pickle.dump(mlp, open(name_model+".sav", 'wb')) 

/home/daniil/program/python/project/layout_highlighter/imgDoc/env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


/home/daniil/program/python/project/layout_highlighter/imgDoc/env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


/home/daniil/program/python/project/layout_highlighter/imgDoc/env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


/home/daniil/program/python/project/layout_highlighter/imgDoc/env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


/home/daniil/program/python/project/layout_highlighter/imgDoc/env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


...

/home/daniil/program/python/project/layout_highlighter/imgDoc/env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


/home/daniil/program/python/project/layout_highlighter/imgDoc/env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


/home/daniil/program/python/project/layout_highlighter/imgDoc/env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


/home/daniil/program/python/project/layout_highlighter/imgDoc/env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


/home/daniil/program/python/project/layout_highlighter/imgDoc/env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [16]:
max_acc

0.22